# Training a classifier for weights of author vision components

The features are the distance of the author to any tweet in the conversation
indicated by the following structures:
- subtree to viewed tweet from a tweet the author wrote
- root closeness of viewed tweet
- time delta to viewed tweet from tweets the author wrote

#### Loading the data from the pickled version
1. importing libraries
2. checking gpu support


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from keras import backend as K
import pickle5 as pickle

is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
print("cuda gpu is available: {}".format(is_cuda_gpu_available))
with open("data/vision_graph_data.pkl", 'rb') as f:
    df = pickle.load(f)

cuda gpu is available: True


2022-08-03 11:48:24.885057: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 11:48:24.885415: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 11:48:24.885647: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 11:48:24.885948: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 11:48:24.886182: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [23]:
def is_not_reddit_or_twitter(text):
    if text == "reddit" or text == "twitter":
        return False
    else:
        return True

# remove non-features
current = df.current
beam_node = df.beam_node
platform = df.platform
platform.value_counts()

# filtering data that is not twitter or reddit
to_delete_rows = platform.apply(lambda x: is_not_reddit_or_twitter(x))
df = df.drop(df[to_delete_rows].index)
df.describe()

,timedelta,root_distance_0,y,current,beam_node,has_followed_path,has_follow_path,conversation_id,reply_distance_2,reply_distance_3,...,reply_distance_40,root_distance_32,root_distance_33,root_distance_34,root_distance_35,root_distance_36,root_distance_37,root_distance_38,root_distance_39,root_distance_40
count,3.419533e+06,3.419533e+06,3.419533e+06,3.419533e+06,3.419533e+06,3419533.0,3419533.0,3.419533e+06,3.419533e+06,3.419533e+06,...,3.419533e+06,3.419533e+06,3.419533e+06,3.419533e+06,3.419533e+06,3.419533e+06,3.419533e+06,3.419533e+06,3.419533e+06,3.419533e+06
mean,4.274285e+04,3.969987e-02,5.875656e-02,1.296209e+18,1.296008e+18,0.0,0.0,1.295209e+18,6.031525e-03,3.544929e-03,...,8.773128e-07,5.848752e-06,5.263877e-06,4.679001e-06,4.094126e-06,2.047063e-06,1.754626e-06,1.462188e-06,1.169750e-06,8.773128e-07
std,1.113152e+06,1.952532e-01,2.351685e-01,5.464027e+17,5.464532e+17,0.0,0.0,5.464034e+17,7.742834e-02,5.943369e-02,...,9.366495e-04,2.418413e-03,2.294308e-03,2.163095e-03,2.023391e-03,1.430755e-03,1.324622e-03,1.209209e-03,1.081550e-03,9.366495e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,3.880000e+02,3.880000e+02,0.0,0.0,1.745030e+05,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.417000e+03,0.000000e+00,0.000000e+00,1.493867e+18,1.493522e+18,0.0,0.0,1.490282e+18,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,9.486000e+03,0.000000e+00,0.000000e+00,1.531780e+18,1.531699e+18,0.0,0.0,1.531571e+18,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.239300e+04,0.000000e+00,0.000000e+00,1.539142e+18,1.539004e+18,0.0,0.0,1.538890e+18,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.148715e+08,1.000000e+00,1.000000e+00,1.552563e+18,1.552563e+18,0.0,0.0,1.552559e+18,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [24]:
from random import sample

# limit the twitter data and the reddit data to the same amount and prevent gpu problem
df_conversations = df[df["platform"]=="twitter"].conversation_id.tolist()
reddit_data_count = df.loc[df.platform == "reddit", 'platform'].count()
twitter_data_count = df.loc[df.platform == "twitter", 'platform'].count()
assert twitter_data_count > reddit_data_count

current_count = 0
n = 1
while current_count < reddit_data_count:
    chosen_conversation_ids = sample(df_conversations, n)
    df_candidate = df[df["conversation_id"].isin(chosen_conversation_ids)]
    n = n + 1
    current_count = df_candidate.shape[0]

print("chosen {} conversations and gotten {} from twitter compared to {} from reddit".format(n, current_count, reddit_data_count))
not_chosen_conversation_ids  = set(df_conversations) - set(chosen_conversation_ids)
df = df[~df["conversation_id"].isin(not_chosen_conversation_ids)]
df.describe()

chosen 242 conversations and gotten 522223 from twitter compared to 514793 from reddit


,timedelta,root_distance_0,y,current,beam_node,has_followed_path,has_follow_path,conversation_id,reply_distance_2,reply_distance_3,...,reply_distance_40,root_distance_32,root_distance_33,root_distance_34,root_distance_35,root_distance_36,root_distance_37,root_distance_38,root_distance_39,root_distance_40
count,1.037016e+06,1.037016e+06,1.037016e+06,1.037016e+06,1.037016e+06,1037016.0,1037016.0,1.037016e+06,1.037016e+06,1.037016e+06,...,1.037016e+06,1.037016e+06,1.037016e+06,1.037016e+06,1.037016e+06,1.037016e+06,1.037016e+06,1.037016e+06,1.037016e+06,1.037016e+06
mean,6.995113e+04,3.706500e-02,7.632669e-02,7.692708e+17,7.691746e+17,0.0,0.0,7.670094e+17,1.332766e-02,7.574618e-03,...,2.892916e-06,1.060736e-05,9.643053e-06,8.678747e-06,7.714442e-06,6.750137e-06,5.785832e-06,4.821526e-06,3.857221e-06,2.892916e-06
std,1.574565e+06,1.889212e-01,2.655202e-01,7.639554e+17,7.638775e+17,0.0,0.0,7.624943e+17,1.146737e-01,8.670208e-02,...,1.700856e-03,3.256878e-03,3.105313e-03,2.945960e-03,2.777479e-03,2.598095e-03,2.405370e-03,2.195793e-03,1.963978e-03,1.700856e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,3.880000e+02,3.880000e+02,0.0,0.0,1.745030e+05,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.532000e+03,0.000000e+00,0.000000e+00,5.004466e+07,5.030167e+07,0.0,0.0,5.112814e+07,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.300200e+04,0.000000e+00,0.000000e+00,1.455657e+18,1.455632e+18,0.0,0.0,9.625765e+17,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.799200e+04,0.000000e+00,0.000000e+00,1.533731e+18,1.533575e+18,0.0,0.0,1.533426e+18,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.505164e+08,1.000000e+00,1.000000e+00,1.552559e+18,1.552544e+18,0.0,0.0,1.552169e+18,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [25]:

# some utility functions to take the columns that are used as features
non_feature_list = ["current", "beam_node", "conversation_id", "platform", "has_followed_path", "has_follow_path"]
def take_features(df):
    df = df.drop(non_feature_list, axis=1)
    return df

def take_non_features(df):
    column_names = df.columns.values
    feature_list = [column_name for column_name in column_names if column_name not in non_feature_list]
    df = df.drop(feature_list, axis=1)
    return df

In [26]:

# training functions

def train_model(df):
    # dropping non-reddit non-twitter data
    df = take_features(df)
    # df = df[df["root_distance_0"] == 0]

    # normalize timedelta (put between 0 and 1)
    dt = df.timedelta
    timedelta_normalized = (dt - dt.min()) / (dt.max() - dt.min())
    df.timedelta = timedelta_normalized

    # selecting train and test datasets
    train, test = train_test_split(df, test_size=0.2)
    train.describe()

    # train the model
    y = train.y
    x = train.drop("y", axis=1)
    print(x.shape)
    print(y.shape)

    # import tensorflow and train the model

    print(tf.__version__)
    input_shape = (x.shape[1],)
    model = Sequential([
        Dense(1, activation='sigmoid', input_shape=input_shape)
    ])

    # stochastic gradient descend as a classifier seem appropriate
    model.compile(
        optimizer='sgd',
        loss='binary_crossentropy',
        metrics=['accuracy', 'mae']
    )

    # model.fit(x, y, epochs=3)
    model.fit(x, y)
    # evaluate the model on the test set
    test_y = test.y
    test_x = test.drop("y", axis=1)

    loss, accuracy, mae = model.evaluate(test_x, test_y)
    print("the accuracy on the training set is {} and the mae is {}".format(accuracy, mae))

    return x, y, test_x, test_y, model


def inspect_model(x, y, test_x, test_y, model):
    # have a look at some prediction
    # reply_distance_2 = test_x[test_x["reply_distance_2"] == 1]
    # first_rows = reply_distance_2.head(2)
    # print(first_rows)
    # model.predict(first_rows)

    # let's have a look at the weights and biases of the hidden layer
    first_layer_weights = model.layers[0].get_weights()[0]
    first_layer_biases = model.layers[0].get_weights()[1]
    # print(first_layer_weights)
    column_names = x.columns.values
    for i in range(len(column_names[:5])):
        print("feature {} has weight {} \n".format(column_names[i], first_layer_weights[i]))


In [27]:
# have a look for reddit
tw_df = df[df["platform"] == "twitter"]
tw_x, tw_y, tw_test_x, tw_test_y, tw_model = train_model(tw_df)

(417778, 81)
(417778,)
2.6.0
3264/3264 [==============================] - 3s 810us/step - loss: 0.0867 - accuracy: 0.9656 - mae: 0.0533

In [28]:
rd_df = df[df["platform"] == "reddit"]
rd_x, rd_y, rd_test_x, rd_test_y, rd_model = train_model(rd_df)

(411834, 81)
(411834,)
2.6.0
3218/3218 [==============================] - 3s 797us/step - loss: 0.3259 - accuracy: 0.8810 - mae: 0.1897

In [29]:
# inspect_model(tw_x, tw_y, tw_test_x, tw_test_y, tw_model)
# inspect_model(rd_x, rd_y, rd_test_x, rd_test_y, rd_model)

In [30]:
# rd_sample_size = rd_df.shape[0]
# tw_df = tw_df.sample(n=rd_sample_size)


tw_non_features = take_non_features(tw_df)
rd_non_features = take_non_features(rd_df)
tw_non_features

,current,beam_node,has_followed_path,has_follow_path,platform,conversation_id
105,1527614293239382016,1527522295354368005,0,0,twitter,1527522295354368005
106,1527614293239382016,1527613879110574081,0,0,twitter,1527522295354368005
107,1527614293239382016,1527612414765801473,0,0,twitter,1527522295354368005
108,1527614293239382016,1527612113010802689,0,0,twitter,1527522295354368005
109,1527614293239382016,1527573751172415488,0,0,twitter,1527522295354368005
...,...,...,...,...,...,...
3419612,1543869722831331329,1543865562958155776,0,0,twitter,1543814358718992388
3419613,1543869722831331329,1543858541068521478,0,0,twitter,1543814358718992388
3419614,1543869722831331329,1543863860259536898,0,0,twitter,1543814358718992388
3419615,1543869722831331329,1543859236878577666,0,0,twitter,1543814358718992388


In [31]:

from util.abusing_lists import batch

tw_features_y = take_features(tw_df)
tw_features = tw_features_y.drop("y", axis=1)
rd_features_y = take_features(rd_df)
rd_features = rd_features_y.drop("y", axis=1)
rd_predictions = rd_model.predict(rd_features)
tw_predictions = tw_model.predict(tw_features)

tw_vision = tw_non_features.assign(predictions=tw_predictions)
rd_vision = rd_non_features.assign(predictions=rd_predictions)
#tw_vision


In [32]:
combined_vision = tw_vision.append(rd_vision)
combined_vision


not_needed_list = ["beam_node", "has_followed_path", "has_follow_path"]
combined_vision = combined_vision.drop(not_needed_list, axis=1)
combined_vision

,current,platform,conversation_id,predictions
105,1527614293239382016,twitter,1527522295354368005,1.0
106,1527614293239382016,twitter,1527522295354368005,1.0
107,1527614293239382016,twitter,1527522295354368005,1.0
108,1527614293239382016,twitter,1527522295354368005,1.0
109,1527614293239382016,twitter,1527522295354368005,1.0
...,...,...,...,...
3418060,68851107,reddit,12994552,1.0
3418061,68851107,reddit,12994552,1.0
3418062,68851107,reddit,12994552,1.0
3418063,68851107,reddit,12994552,1.0


In [33]:
gpm = combined_vision.groupby(["platform", "conversation_id", "current"]).mean()
gpm

predictions
platform conversation_id     current                         
reddit   174503              4541493                 0.987631
                             5662402                 1.000000
                             8711684                 0.926990
                             10143803                1.000000
                             11735997                0.979709
...                                                       ...
twitter  1552168511165894657 1552397547590795264     1.000000
                             1552413538924249090     1.000000
                             1552421341940387844     1.000000
                             1552544216697430017     1.000000
                             1552559062730113025     1.000000

[38437 rows x 1 columns]

In [34]:
gpm_per_conversation = gpm.groupby(by=["platform", "conversation_id"]).mean()
gpm_per_conversation

predictions
platform conversation_id                 
reddit   174503                  0.984993
         203904                  0.977508
         209098                  0.990180
         313699                  0.973880
         471878                  0.957565
...                                   ...
twitter  1544585020161101824     0.986020
         1545721821647060993     0.998108
         1551481579784085504     0.999732
         1552024342661251072     0.995596
         1552168511165894657     0.996841

[1301 rows x 1 columns]

In [35]:
gpm_per_platform = gpm.groupby(by=["platform"]).mean()
gpm_per_platform



,predictions
platform,
reddit,0.968935
twitter,0.990445


In [36]:
combined_vision = combined_vision.drop(["predictions"], axis=1)


In [37]:
author_count_columns = ["current", "conversation_id", "platform", "y"]
author_df = df[author_count_columns]
author_df

,current,conversation_id,platform,y
105,1527614293239382016,1527522295354368005,twitter,1
106,1527614293239382016,1527522295354368005,twitter,0
107,1527614293239382016,1527522295354368005,twitter,0
108,1527614293239382016,1527522295354368005,twitter,0
109,1527614293239382016,1527522295354368005,twitter,0
...,...,...,...,...
3419612,1543869722831331329,1543814358718992388,twitter,0
3419613,1543869722831331329,1543814358718992388,twitter,1
3419614,1543869722831331329,1543814358718992388,twitter,0
3419615,1543869722831331329,1543814358718992388,twitter,0


In [38]:
author_df = author_df.groupby(["platform", "conversation_id", "current"]).sum()
author_df

y
platform conversation_id     current                
reddit   174503              4541493               2
                             5662402               5
                             8711684               4
                             10143803              2
                             11735997             11
...                                               ..
twitter  1552168511165894657 1552397547590795264   1
                             1552413538924249090   1
                             1552421341940387844   1
                             1552544216697430017   1
                             1552559062730113025   1

[38437 rows x 1 columns]

In [39]:
author_df = author_df.groupby(["platform", "conversation_id"]).mean()
author_df


y
platform conversation_id              
reddit   174503               4.244444
         203904               2.810811
         209098               1.875000
         313699               1.230769
         471878               3.285714
...                                ...
twitter  1544585020161101824  4.950820
         1545721821647060993  1.217949
         1551481579784085504  2.360000
         1552024342661251072  4.300000
         1552168511165894657  1.740260

[1301 rows x 1 columns]

In [40]:
distinct_view_columns = ["current", "conversation_id", "platform"]
distinct_view_df = df[distinct_view_columns]
distinct_view_df

,current,conversation_id,platform
105,1527614293239382016,1527522295354368005,twitter
106,1527614293239382016,1527522295354368005,twitter
107,1527614293239382016,1527522295354368005,twitter
108,1527614293239382016,1527522295354368005,twitter
109,1527614293239382016,1527522295354368005,twitter
...,...,...,...
3419612,1543869722831331329,1543814358718992388,twitter
3419613,1543869722831331329,1543814358718992388,twitter
3419614,1543869722831331329,1543814358718992388,twitter
3419615,1543869722831331329,1543814358718992388,twitter


In [41]:
distinct_views = distinct_view_df.groupby(["current", "conversation_id", "platform"]).size().to_frame('size')
distinct_views = distinct_views.groupby(["platform", "conversation_id"]).mean()
distinct_views


size
platform conversation_id          
reddit   174503               23.0
         203904               19.0
         209098                8.5
         313699                7.0
         471878               11.0
...                            ...
twitter  1544585020161101824  32.0
         1545721821647060993  39.5
         1551481579784085504  25.5
         1552024342661251072  45.5
         1552168511165894657  39.0

[1301 rows x 1 columns]

In [42]:
joined_author_stats = author_df.join(distinct_views)
joined_author_stats = joined_author_stats["y"]/joined_author_stats["size"]
joined_author_stats

platform  conversation_id    
reddit    174503                 0.184541
          203904                 0.147937
          209098                 0.220588
          313699                 0.175824
          471878                 0.298701
                                   ...   
twitter   1544585020161101824    0.154713
          1545721821647060993    0.030834
          1551481579784085504    0.092549
          1552024342661251072    0.094505
          1552168511165894657    0.044622
Length: 1301, dtype: float64

In [43]:
repetition_probability = joined_author_stats.groupby("platform").mean()
repetition_probability

platform
reddit     0.229424
twitter    0.046573
dtype: float64